In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
import warnings

In [2]:
import xgboost as xgb

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def read_ec_data(filename):
    with open(filename) as fid:
        data_dct = defaultdict(dict)
        for line in fid:
            fields = line.strip('\n').split('\t')
            if fields[1] == 'SLP':
                continue
            ec_time = datetime.strptime(fields[0], '%Y%m%d%H')
            forecast_time = (ec_time + timedelta(hours=12))
            for idx in range(-12, 24):
                data_dct[forecast_time][f'{fields[1]}.{idx}'] = float(fields[idx + 12 + 2])
    return pd.DataFrame(data_dct).transpose()

In [8]:
def read_obs(filename):
    obs_data = pd.read_csv(filename, header=None, names=['time', 'obs'], sep='\t')
    obs_data['date'] = pd.to_datetime(obs_data['time'] // 10000, format='%Y%m%d')
    obs_data['hour'] = obs_data['time'] % 100
    obs_data2 = obs_data.pivot(columns='hour', index='date', values='obs')
    return obs_data2

In [6]:
ec1 = read_ec_data('data/ec_fcst_2018030112_2018103112.txt')
ec2 = read_ec_data('data/ec_fcst_2018110112_2018123012.txt')
ec = pd.concat([ec1, ec2], axis=0)

In [10]:
obs_p1 = read_obs('data/dir_01_2018030112_2018103112.txt')
obs_p2 = read_obs('data/dir_01_2018110112_2018123112.txt')
obs = pd.concat([obs_p1, obs_p2], axis=0).resample('1D').mean()

In [98]:
%matplotlib inline

In [97]:
obs.head()

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
2018-03-02,103.0,103.0,171.0,127.0,234.0,237.0,134.0,188.0,187.0,152.0,...,28.0,39.0,147.0,318.0,70.0,26.0,28.0,360.0,1.0,8.0
2018-03-03,348.0,355.0,43.0,107.0,173.0,160.0,111.0,83.0,117.0,126.0,...,303.0,315.0,155.0,41.0,48.0,45.0,59.0,26.0,61.0,138.0
2018-03-04,158.0,162.0,172.0,159.0,167.0,152.0,159.0,146.0,142.0,155.0,...,78.0,153.0,82.0,64.0,66.0,37.0,33.0,282.0,166.0,115.0
2018-03-05,54.0,99.0,136.0,172.0,164.0,189.0,174.0,197.0,178.0,168.0,...,128.0,82.0,113.0,119.0,114.0,92.0,46.0,72.0,98.0,84.0
2018-03-06,73.0,116.0,143.0,149.0,137.0,134.0,182.0,188.0,168.0,162.0,...,153.0,154.0,160.0,153.0,148.0,145.0,112.0,152.0,136.0,151.0


In [12]:
yesterday_obs = obs.shift(1)
yesterday_obs = yesterday_obs[[x for x in range(12, 24)]]

In [13]:
yesterday_obs.columns = [x - 24 for x in yesterday_obs.columns]

In [14]:
obs_mat = pd.concat([yesterday_obs, obs], axis=1)
obs_mat.columns = [f'obs.{x}' for x in obs_mat.columns]

In [15]:
obs_mat = obs_mat.loc[obs_mat['obs.-12'].notnull()]

In [16]:
raw_data = ec.merge(obs_mat, left_index=True, right_index=True)

In [28]:
u = 0
v = -1

In [29]:
np.arctan2(-u, -v) / np.pi * 180 % 360

0.0

### 特征加工

In [41]:
# 风速，湿球温度与气温差值，预报风速误差
for idx in range(-12, 24):
    raw_data[f'ws.{idx}'] = np.sqrt(raw_data[f'U10.{idx}'] ** 2 + raw_data[f'V10.{idx}'] ** 2)
    raw_data[f'wd.{idx}'] = np.arctan2(-raw_data[f'U10.{idx}'], -raw_data[f'V10.{idx}']) / np.pi * 180 % 360
    raw_data[f'rh_delta.{idx}'] = raw_data[f'T.{idx}'] - raw_data[f'RH.{idx}']
    raw_data[f'bias.{idx}'] = (raw_data[f'obs.{idx}'] - raw_data[f'wd.{idx}'] + 180) % 360 - 180

In [46]:
# 气压变，温度变，风速变
for idx in range(0, 24):
    for span in (1, 3, 6, 12):
        raw_data[f'PSFC_{span}d.{idx}'] = raw_data[f'PSFC.{idx}'] - raw_data[f'PSFC.{idx-span}']
        raw_data[f'T_{span}d.{idx}'] = raw_data[f'T.{idx}'] - raw_data[f'T.{idx-span}']
        raw_data[f'ws_{span}d.{idx}'] = raw_data[f'ws.{idx}'] - raw_data[f'ws.{idx-span}']
        raw_data[f'wd_{span}d.{idx}'] = (raw_data[f'wd.{idx}'] - raw_data[f'wd.{idx-span}'] + 180) % 360 - 180

In [32]:
is_train = raw_data.index < datetime(2018, 11, 1)
is_eval = (raw_data.index >= datetime(2018, 11, 1)) & (raw_data.index < datetime(2018, 12, 1))
is_test = raw_data.index >= datetime(2018, 12, 1)

In [33]:
is_train.sum(), is_eval.sum(), is_test.sum()

(243, 30, 31)

### 训练模型

每个预报时次独立训练。预测对象为实际风向与EC风向的差值。预报结果再叠加上EC风向作为最终的预报结果。

In [34]:
def rmse(y_arr):
    return np.sqrt((y_arr ** 2).mean())

In [79]:
fc_hr = 6

In [80]:
feat_list = [f'U10.{x}' for x in range(-12, 24)] + [f'V10.{x}' for x in range(-12, 24)] + \
    [f'bias.{x}' for x in range(-12, 0)] + [f'ws.{x}' for x in range(-12, 24)] + \
    [f'wd.{x}' for x in range(-12, 24)] + \
    [f'rh_delta.{x}' for x in range(-12, 24)] + \
    [f'PSFC_{span}d.{fc_hr}' for span in (1, 3, 6, 12)] + [f'T_{span}d.{fc_hr}' for span in (1, 3, 6, 12)] + \
    [f'ws_{span}d.{fc_hr}' for span in (1, 3, 6, 12)] + [f'wd_{span}d.{fc_hr}' for span in (1, 3, 6, 12)]

In [81]:
x_train = raw_data.loc[is_train, feat_list]
x_eval = raw_data.loc[is_eval, feat_list]
x_test = raw_data.loc[is_test, feat_list]

In [82]:
y_train = raw_data.loc[is_train, f'bias.{fc_hr}']
y_eval = raw_data.loc[is_eval, f'bias.{fc_hr}']
y_test = raw_data.loc[is_test, f'bias.{fc_hr}']

In [83]:
print(rmse(y_train), rmse(y_eval), rmse(y_test))

55.44994712242353 41.42085589921957 57.03474864682167


In [84]:
len(feat_list)

208

### 贝叶斯调参

需要多次迭代，以及计算贝叶斯概率。耗时会成倍增加

In [85]:
from bayes_opt import BayesianOptimization

In [86]:
def bo_result_to_xgb(bo_res):
    xgb_params = bo_res.copy()
    if 'log_gamma' in xgb_params:
        xgb_params['gamma'] = 10**xgb_params['log_gamma']
        xgb_params.pop('log_gamma')
    if 'max_depth' in xgb_params:
        xgb_params['max_depth'] = int(np.round(xgb_params['max_depth']))
    if 'max_delta_step' in xgb_params:
        xgb_params['max_delta_step'] = int(np.round(xgb_params['max_delta_step']))
    if 'subsample' in xgb_params:
        xgb_params['subsample'] = max(min(xgb_params['subsample'], 1), 0)
    if 'colsample_bytree' in xgb_params:
        xgb_params['colsample_bytree'] = max(min(xgb_params['colsample_bytree'], 1), 0)
    return xgb_params

In [87]:
def xgb_model(**kwargs):
    xgb_params = bo_result_to_xgb(kwargs)
    clf = xgb.XGBRegressor(booster='gbtree', n_estimators=100, verbosity=0, n_jobs=16, seed=42,
                            reg_alpha=0.1, reg_lambda=0.1, **xgb_params)
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_eval, y_eval)], eval_metric='rmse',
            verbose=False)
    eval_result = clf.evals_result()
    train_rmse = eval_result['validation_0']['rmse'][-1]
    dev_rmse = eval_result['validation_1']['rmse'][-1]
    n_trees = len(eval_result['validation_0']['rmse'])
    return -dev_rmse

In [88]:
xgb_bayes = BayesianOptimization(xgb_model, {
    'learning_rate': (0.02, 0.06),
    'max_depth': (3, 7),
    'log_gamma': (-3, 1),
    'min_child_weight': (0, 20),
    'max_delta_step': (0, 10),
    'subsample': (0.3, 0.9),
    'colsample_bytree': (0.3, 0.9)
})

In [89]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    xgb_bayes.maximize(init_points=15, n_iter=25)

|   iter    |  target   | colsam... | learni... | log_gamma | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -43.53    |  0.5627   |  0.05434  | -2.996    |  9.497    |  3.418    |  10.26    |  0.6865   |
|  2        | -44.99    |  0.5148   |  0.03072  | -1.131    |  1.662    |  6.266    |  7.78     |  0.3752   |
|  3        | -44.25    |  0.3934   |  0.02347  | -2.483    |  2.105    |  5.189    |  18.86    |  0.3469   |
|  4        | -43.23    |  0.5141   |  0.03069  |  0.4306   |  2.554    |  6.903    |  4.784    |  0.6128   |
|  5        | -47.0     |  0.3896   |  0.05388  | -0.223    |  7.275    |  3.759    |  6.651    |  0.6772   |
|  6        | -42.18    |  0.3204   |  0.0558   | -2.643    |  0.5885   |  5.25     |  9.117    |  0.3563   |
|  7        | -44.18    |  0.4304   |  0.02794  |  0.9634   |  5.52     |  4.983    |  4.266    |  0.6723   |
|  8      

In [90]:
best_params = xgb_bayes.max['params']
xgb_params = bo_result_to_xgb(best_params)

In [91]:
clf = xgb.XGBRegressor(booster='gbtree', n_estimators=100, verbosity=0, n_jobs=16, seed=42,
                            reg_alpha=0.1, reg_lambda=0.1, **xgb_params)

In [92]:
clf.fit(x_train, y_train, eval_set=[(x_eval, y_eval)], eval_metric='rmse', verbose=20)

[0]	validation_0-rmse:41.6592
[20]	validation_0-rmse:42.0863
[40]	validation_0-rmse:41.9187
[60]	validation_0-rmse:41.269
[80]	validation_0-rmse:41.3623
[99]	validation_0-rmse:41.7614


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5871745279884921, gamma=0.0042838197232680465,
       learning_rate=0.02272420122094703, max_delta_step=3, max_depth=5,
       min_child_weight=4.4429050610784575, missing=None, n_estimators=100,
       n_jobs=16, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=1, seed=42,
       silent=True, subsample=0.7912238288974662, verbosity=0)

验证集最终的指标与贝叶斯优化时最佳指标理应一致

In [93]:
y_pred_eval = clf.predict(x_eval)
y_pred_eval = pd.Series(y_pred_eval, index=y_eval.index)

In [94]:
rmse(y_pred_eval - y_eval)

41.76142926691161

In [95]:
y_test_eval = clf.predict(x_test)
y_test_eval = pd.Series(y_test_eval, index=y_test.index)

In [96]:
rmse(y_test - y_test_eval)

55.08757815640879